<a href="https://www.kaggle.com/code/ritvik1909/neural-machine-translation-attention" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Neural Machine Translation

In this notebook we will implement a small transformer model for machine translation task. Our model would be able to translate human readable dates in any format to YYYY-MM-DD format.

We will be using `faker` module to generate our dataset

In [ ]:
%%capture
!pip install -q faker

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import re, random

from faker import Faker
from babel.dates import format_date

pd.options.display.max_colwidth = None
sns.set_style('darkgrid')

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import losses, callbacks, utils, models, Input
from tensorflow.keras import layers as L

# Data Generation

In [ ]:
# Constants
class config():  
    SAMPLE_SIZE = 10_00_000        
    DATE_FORMATS = [
        'short', 'medium', 'long', 'full',
        'd MMM YYY', 'd MMMM YYY', 'dd/MM/YYY',
        'EE d, MMM YYY', 'EEEE d, MMMM YYY', 'd of MMMM YYY',
    ]
    VALIDATION_SIZE = 0.1
    BATCH_SIZE = 32
    MAX_EPOCHS = 25
    EMBED_DIM = 16
    DENSE_DIM = 16
    NUM_HEADS = 2
    X_LEN = 30
    Y_LEN = 14
    NUM_ENCODER_TOKENS = 35
    NUM_DECODER_TOKENS = 14

In [ ]:
faker = Faker()
print('Sample dates for each format\n')

for fmt in set(config.DATE_FORMATS):
    print(f'{fmt:20} =>  {format_date(faker.date_object(), format=fmt, locale="en")}')

In [ ]:
# a utility data cleaning function
def clean_date(raw_date):
    return raw_date.lower().replace(',', '')

# this function will generate our data in a data frame
def create_dataset(num_rows):
    dataset = []
    
    for i in tqdm(range(num_rows)):
        dt = faker.date_object()
        for fmt in config.DATE_FORMATS:
            try:
                date = format_date(dt, format=fmt, locale='en')
                human_readable = clean_date(date)
                machine_readable = f"@{dt.isoformat()}#" # adding a start token '@' and end token '#'
            except AttributeError as e:
                date = None
                human_readable = None
                machine_readable = None
            if human_readable is not None and machine_readable is not None:
                dataset.append((human_readable, machine_readable))
 
    return pd.DataFrame(dataset, columns=['human_readable', 'machine_readable'])

In [ ]:
# generate the dataset using the function defined above
dataset = create_dataset(config.SAMPLE_SIZE)
dataset  = dataset.drop_duplicates(subset=['human_readable']).sample(frac=1.0).reset_index(drop=True)
print(dataset.shape)
dataset.head()

**Define tokenizers for both the languages (human readable and machine readable dates)**

In [ ]:
human_tokenizer = Tokenizer(char_level=True)
machine_tokenizer = Tokenizer(char_level=True)

human_tokenizer.fit_on_texts(dataset['human_readable'].values)
machine_tokenizer.fit_on_texts(dataset['machine_readable'].values)

print(human_tokenizer.word_index)
print(machine_tokenizer.word_index)

In [ ]:
# A utility function to clean and tokenize the text and then pad the sequence
def preprocess_input(date, tokenizer, max_len):
    seq = [i[0] for i in tokenizer.texts_to_sequences(date.lower().replace(',', ''))]
    seq = pad_sequences([seq], padding='post', maxlen=max_len)[0]
    return seq

**Preprocessing the data**

In [ ]:
%%time

X = np.array(list(map(lambda x: preprocess_input(x, human_tokenizer, config.X_LEN), dataset['human_readable'])))
y = np.array(list(map(lambda x: preprocess_input(x, machine_tokenizer, config.Y_LEN), dataset['machine_readable'])))

X.shape, y.shape

In [ ]:
# Cross Validation
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=config.VALIDATION_SIZE, random_state=19)

A utility function to generate batches of data
* encoder input data would be source language text
* decoder input data would be target language text
* decoder output data would be target language text shifted one time stamp forward (one hot encoded)

In [ ]:
def generate_batch(X, y, batch_size=config.BATCH_SIZE):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = X[j:j+batch_size]
            decoder_input_data = y[j:j+batch_size]
            output = y[j:j+batch_size]
            decoder_output_data = np.zeros_like(output)
            decoder_output_data[:,:-1] = output[:, 1:]
            decoder_target_data = utils.to_categorical(decoder_output_data, num_classes=config.NUM_DECODER_TOKENS)
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

# Model

Let's define the key components for our model
* Positional Embedding Layer
* Encoder Block
* Decoder Block

In [ ]:
class PositionalEmbedding(L.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = L.Embedding(input_dim, output_dim)
        self.position_embeddings = L.Embedding(sequence_length, output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim
        
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config
    
class TransformerEncoder(L.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = L.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential([L.Dense(dense_dim, activation='relu'), L.Dense(embed_dim)])
        self.layernorm1 = L.LayerNormalization()
        self.layernorm2 = L.LayerNormalization()
    
    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[: tf.newaxis, :]
        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm2(proj_input + proj_output)
    
    def get_config(self):
        config = super().get_confog()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim
        })
        return config    
    
class TransformerDecoder(L.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = L.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = L.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential([
            L.Dense(dense_dim, activation='relu'), L.Dense(embed_dim)
        ])
        self.layernorm_1 = L.LayerNormalization()
        self.layernorm_2 = L.LayerNormalization()
        self.layernorm_3 = L.LayerNormalization()
        self.support_masking = False
        
    def get_config(self):
        config = super().get_config()
        config.update({
            'embed_dim': self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config
    
    def call(self, inputs, encoder_outputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=None
        )
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1, value=encoder_outputs, key=encoder_outputs, attention_mask=None
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2
        )
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

Lets Define our model

* Positional Embedding is applied to Encoder Inputs
* Embedded Encoder Inputs are fed to the Transformer Encoder
* Positional Embedding is applied to Decoder Inputs 
* Encoder Outputs and along with the Embedded Decoder Inputs are fed to the transformer Decoder
* We optionally apply dropout to the outputs from the decoder block before feeding it to the output layer

In [ ]:
encoder_inputs = keras.Input(shape=(None, ), dtype="int64", name="human")
x = PositionalEmbedding(config.X_LEN, config.NUM_ENCODER_TOKENS, config.EMBED_DIM)(encoder_inputs)
encoder_outputs = TransformerEncoder(config.EMBED_DIM, config.DENSE_DIM, config.NUM_HEADS)(x)

decoder_inputs = keras.Input(shape=(None, ), dtype="int64", name="machine" )
x = PositionalEmbedding(config.Y_LEN, config.NUM_DECODER_TOKENS, config.EMBED_DIM)(decoder_inputs)
x = TransformerDecoder(config.EMBED_DIM, config.DENSE_DIM, config.NUM_HEADS)(x, encoder_outputs)
x = L.Dropout(0.5)(x)
decoder_outputs = L.Dense(config.NUM_DECODER_TOKENS, activation="softmax")(x)

transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
transformer.compile(optimizer="adam", loss="categorical_crossentropy", metrics="accuracy")

transformer.summary()
utils.plot_model(transformer, show_shapes=True, expand_nested=True)

In [ ]:
es = callbacks.EarlyStopping(monitor="val_loss", patience=3, verbose=1, restore_best_weights=True, min_delta=1e-4)
rlp = callbacks.ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1)

In [ ]:
%%capture training_log

history = transformer.fit(
    generate_batch(X_train, y_train), steps_per_epoch = np.ceil(len(X_train)/config.BATCH_SIZE),
    validation_data=generate_batch(X_valid, y_valid), validation_steps=np.ceil(len(X_valid)/config.BATCH_SIZE),
    epochs=config.MAX_EPOCHS, callbacks=[es, rlp], 
)

In [ ]:
with open('training.log', 'w') as f: f.write(training_log.stdout)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 6))
pd.DataFrame(history.history).plot(ax=ax)
del history

# Evaluation

**Lets see our model in action**

In [ ]:
# this function converts the token ids to token text
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return ""

# this function is used to generate predictions
# target sequence is intialised as the start token
# then predictions are taken from the model iteratively 
def predict_sequence(source):
    target_seq = "@"
    for i in range(14):
        tok_target_seq = preprocess_input(target_seq, machine_tokenizer, config.Y_LEN).reshape(1, -1)
        output_tokens = transformer.predict([source, tok_target_seq])
        token_index = np.argmax(output_tokens[0, i, :])
        token = word_for_id(token_index, machine_tokenizer)
        target_seq += token
        
    return target_seq

# this function decodes the complete sequence of token ids into text
def decode_sequence(tokenizer, source):
    target = list()
    for i in source:
        word = word_for_id(i, tokenizer)
        if word is None:
            break
        target.append(word)
    return ''.join(target)

In [ ]:
query_text = 'saturday 19 september 1998'
query = np.array(list(map(lambda x: preprocess_input(x, human_tokenizer, config.X_LEN), [query_text])))
print('SOURCE     :', query_text)
print('PREDICTION :', predict_sequence(query))